In [117]:
import pandas as pd
import parser
import imp



from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
from langdetect import detect
import requests
from requests.exceptions import MissingSchema, ReadTimeout, ConnectTimeout, ConnectionError
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 50
imp.reload(parser)

<module 'parser' from '/home/niwuro/Projects/NCO/npo_classification/parser.py'>

In [2]:
# !git commit -a -m "minor fix"

In [118]:
dataset = pd.read_excel("np_ngo_classification.xlsx")
dataset.head(1)

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Ссылка на соц сеть,Вид деятельности 1,...,Вид деятельности 3,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12
0,Москва,Межрегиональная общественная благотворительная...,5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-...",0,0,"http://greenstartup.ru/, http:/9911850.ru/",https://princip1.livejournal.com/,3_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
# Drop empty rows
sites = dataset[~dataset["Ссылка на сайт"].isnull()]
sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
social = dataset[~dataset["Ссылка на соц сеть"].isnull()]

# Merging social web-page to a web-site, adding new data samples 
social["Ссылка на сайт"] = social["Ссылка на соц сеть"]
social.drop(columns=["Ссылка на соц сеть"], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True)

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [120]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(parser.link_splitter)
dataset.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
concat_dataset["num_sites"] = concat_dataset["Ссылка на сайт"].apply(len)

In [121]:
concat_dataset

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 4,Вид деятельности 5,Вид деятельности 6,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites
0,Москва,Межрегиональная общественная благотворительная...,5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-...",0,0,"[http://greenstartup.ru/, http:/9911850.ru/]",3_2,6_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,Москва,Межрегиональная благотворительная общественная...,7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49...",1,1,[http://oppid.ru/],2_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Москва,Некоммерческая организация Благотворительный ф...,7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,[capital-mercy.ru],1_9,1_2,...,2_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Москва,Российский общественный благотворительный фонд...,7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская,...",1,1,[http://www.rbfondveteranov.ru/],1_4,5_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,Москва,Благотворительный фонд поддержки и реализации ...,7704278848,2011-09-30,"г Москва, р-н Хамовники, пер Знаменский Б., д ...",0,0,[http://www.avccharity.com/],4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,[http://gergiev-charity.com/rus/],4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,Москва,Международный Благотворительный Фонд Владимира...,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, ...",1,0,[http://spivakov.ru/fund/contacts/],4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,Москва,Благотворительный фонд поддержки детей с особе...,7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский ...",1,1,[http://yaest.ru/],1_7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,Москва,Благотворительный фонд помощи бездомным животн...,7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, ...",1,0,[http://fond-kovcheg.ru],6_4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,Москва,Благотворительный фонд поддержки детей имени и...,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА,...",1,1,[http://www.vfonde.ru/],1_7,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [82]:
# ! WARNING: This cell needs to be executed 3 times!
# TODO: 3 times-loop
max_len = len(concat_dataset)
for index, row in concat_dataset.iterrows():
    if len(row["Ссылка на сайт"])>1:
        sites = row["Ссылка на сайт"]
        last_site = sites.pop(-1)
        concat_dataset["Ссылка на сайт"].loc[index] = sites
        row["Ссылка на сайт"] = [last_site]
        concat_dataset.loc[max_len] = row
        max_len += 1

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [161]:
concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(lambda lst: lst[0].strip())

In [163]:
# Fixing mistakes
concat_dataset["Ссылка на сайт"].loc[62] = "http://арктика-надежда.рф" 
concat_dataset["Ссылка на сайт"].loc[154] = "http://9911850.ru/"

/home/tonko22/.virtualenvs/semantic_env_36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [165]:
concat_dataset["web_text"] = concat_dataset["Ссылка на сайт"].apply(parser.get_text_contents)

/home/tonko22/PycharmProjects/npo_classification/parser.py:45: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 45 of the file /home/tonko22/PycharmProjects/npo_classification/parser.py. To get rid of this warning, pass the additional argument 'features="html5lib"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)
/home/tonko22/PycharmProjects/npo_classification/parser.py:59: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this war

In [171]:
concat_dataset = concat_dataset.dropna(subset=["web_text"])
concat_dataset = concat_dataset[concat_dataset['web_text'].astype(str) != '[]']
concat_dataset = concat_dataset.reset_index().drop("index", axis=1)

In [174]:
concat_dataset["web_text"] = concat_dataset["web_text"].apply(lambda x: " ".join(x))

In [175]:
concat_dataset["web_text"]

0      function set_cookie(){var now = new Date();var time = now.getTime();time += 19360000 * 1000;now....
1      html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-tra...
2      html Благотворительный фонд Столица милосердия [if IE 6]>\n<link rel='stylesheet' type='text/css...
3      html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dt...
4      html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd" Благ...
5      html Контактная информация и реквизиты | Фонд Спивакова [if lt IE 9]>\n<script src="http://spiva...
6      html Главная | Я есть <script src="http://ajax.googleapis.com/ajax/libs/jquery/1.12.1/jquery.min...
7      html Фонд «Ковчег» | Поможем вместе window._wpemojiSettings = {"baseUrl":"http:\/\/s.w.org\/imag...
8      html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd" Áëàãîòâîð...
9      html if(!window.BX)window.BX={

In [176]:
concat_dataset.to_pickle(path="np_ngo_classification_links.pkl")

In [177]:
concat_dataset["web_text"].loc[22]

'html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd" if(!window.BX)window.BX={message:function(mess){if(typeof mess==\'object\') for(var i in mess) BX.message[i]=mess[i]; return true;}}; Ак барс созидание <!--\nfunction MM_swapImgRestore() { //v3.0\n  var i,x,a=document.MM_sr; for(i=0;a&&i<a.length&&(x=a[i])&&x.oSrc;i++) x.src=x.oSrc;\n}\n\nfunction MM_preloadImages() { //v3.0\n  var d=document; if(d.images){ if(!d.MM_p) d.MM_p=new Array();\n    var i,j=d.MM_p.length,a=MM_preloadImages.arguments; for(i=0; i<a.length; i++)\n    if (a[i].indexOf("#")!=0){ d.MM_p[j]=new Image; d.MM_p[j++].src=a[i];}}\n}\n\nfunction MM_findObj(n, d) { //v4.01\n  var p,i,x;  if(!d) d=document; if((p=n.indexOf("?"))>0&&parent.frames.length) {\n    d=parent.frames[n.substring(p+1)].document; n=n.substring(0,p);}\n  if(!(x=d[n])&&d.all) x=d.all[n]; for (i=0;!x&&i<d.forms.length;i++) x=d.forms[i][n];\n  for(i=0;!x&&d.layers&&i<d.layers.length;i++) x=MM_

In [6]:
mapping = pd.read_excel("np_ngo_mappings.xlsx")
mapping

In [2]:
dataset["links"] = dataset["Ссылка на сайт"].apply(parser.get_links)

NameError: name 'dataset' is not defined

In [2]:
df = pd.read_pickle('np_ngo_classification_links.pkl')
df['Ссылка на сайт']

0                                http://greenstartup.ru/
1                                       http://oppid.ru/
2                                       capital-mercy.ru
3                         http://www.rbfondveteranov.ru/
4                        http://gergiev-charity.com/rus/
5                      http://spivakov.ru/fund/contacts/
6                                       http://yaest.ru/
7                                 http://fond-kovcheg.ru
8                                  http://www.vfonde.ru/
9                              http://www.tppdetfond.ru/
10                                 http://www.kmfond.ru/
11                          http://mikhailrudyakfund.ru/
12                                       http://okrm.ru/
13                             http://parkgeroev1945.ru/
14                                http://sobr-zvezda.ru/
15                           http://osobyipeterburg.com/
16                                https://vityaz.spb.ru/
17                             

In [3]:
def get_links2(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    try:
        url = url[:-1] if url[-1]=='/' else url
        links = []
        domain = url.split(".")[1] if url.find('www')>0 else url.split(".")[0]
        top_domain = url.split(".")[2] if url.find('www')>0 else url.split(".")[1]
        r = requests.get(url, timeout=(5.05, 35), headers=headers)
        html = r.text
        soup = BeautifulSoup(html, features="lxml")
        for link in soup.findAll('a'):
            ref = link.get('href')
            try:
                if ref[1]=='/' and len(ref)>2:
                    links.append(domain + '.' + top_domain + ref[1:])
                elif ref[0]=='/' and ref[1]!='/':
                    links.append(domain + '.' + top_domain + ref)
                elif ref.find(domain)>0:
                    links.append(ref)
                elif ref.startswith('../'):
                    links.append(domain + '.' + top_domain +  ref[2:])
                elif ref.find('html')>0 and ref[0]!='/':
                    links.append(domain + '.' + top_domain + '/' + ref)
                elif ref.find('html')>0 and ref[0]=='/':
                    links.append(domain + '.' + top_domain + ref)
                else:
                    links.append(ref)
            except:
                pass 
        return list(set(links))
    
    except BaseException as e:
        return print(e)

In [4]:
df.loc[df['Ссылка на сайт'] == 'capital-mercy.ru', 'Ссылка на сайт'] =  'http://capital-mercy.ru'
df.loc[df['Ссылка на сайт'] == 'бфурал.рф', 'Ссылка на сайт'] =  'http://бфурал.рф'
df.loc[df['Ссылка на сайт'] == 'рука-помощи.рф', 'Ссылка на сайт'] =  'http://рука-помощи.рф'

In [5]:
df["links"] = df["Ссылка на сайт"].apply(get_links2)

HTTPSConnectionPool(host='vk.com', port=443): Max retries exceeded with url: /fundrd (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f497d1ebbe0>: Failed to establish a new connection: [Errno -2] Name or service not known',))


In [33]:
df["links"]

0                                                     []
1      [http://oppid.ru/zasedanie-rabochey-gruppyi-po...
2      [http://capital-mercy.ru/fotogalereya/album/de...
3      [http://www.rbfondveteranov.ru/tyrizm, rbfondv...
4      [http://gergiev-charity.com/rus/rus/contact/, ...
5      [http://spivakov.ru/fund/contacts/events/, mai...
6      [about/news/festival-molodezhi-i-studentov, ht...
7      [http://fond-kovcheg.ru/view/2480, http://fond...
8      [http://www.vfonde.ru/index.php?vfonde=program...
9      [tppdetfond.ru/o-fonde/popechitelskiy-sovet/, ...
10     [kmfond.ru/index.php/novosti, kmfond.ru/index....
11     [http://mikhailrudyakfund.ru/#contacts, #t-car...
12     [http://okrm.ru/kak-poluchat-zemlyu/, https://...
13     [http://parkgeroev1945.ru/video/, http://parkg...
14     [http://sobr-zvezda.ru/%d0%b7%d0%b2%d0%b5%d0%b...
15     [http://osobyipeterburg.com/navigacziya/vacanc...
16     [https://vityaz.spb/blagotvoritelnost, https:/...
17     [http://instagram.com/an

In [45]:
def clean_links(url, links):
    domain = url.split(".")[1] if url.find('www')>0 else url.split(".")[0]
    if isinstance(links, list):
        return [l for l in links if domain in l]

In [99]:
df2 = df.apply(lambda row: clean_links(row['Ссылка на сайт'], row['links']),axis=1)

In [100]:
new_df = pd.concat([df, df2], axis=1)

In [101]:
new_df

,Регион,Название организации,ИНН,Год регистрации,Адресс,Возможность пожертвовать деньги на сайте,Возможность обратиться за помощью,Ссылка на сайт,Вид деятельности 1,Вид деятельности 2,...,Вид деятельности 7,Вид деятельности 8,Вид деятельности 9,Вид деятельности 10,Вид деятельности 11,Вид деятельности 12,num_sites,web_text,links,0
0,Москва,Межрегиональная общественная благотворительная...,5032173157,2007-07-12,"г Москва, Орехово-Борисово Южное р-н, МКАД 23-...",0,0,http://greenstartup.ru/,3_2,6_1,...,NaN,NaN,NaN,NaN,NaN,NaN,2,function set_cookie(){var now = new Date();var...,[],[]
1,Москва,Межрегиональная благотворительная общественная...,7702370774,2007-08-28,"г Москва, Мещанский р-н, ул Трифоновская, д 49...",1,1,http://oppid.ru/,2_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD XHTML 1.0 Transitiona...",[http://oppid.ru/zasedanie-rabochey-gruppyi-po...,[http://oppid.ru/zasedanie-rabochey-gruppyi-po...
2,Москва,Некоммерческая организация Благотворительный ф...,7702472455,2014-02-11,"г Москва, Мещанский р-н, ул Щепкина, д 9 стр 1",1,1,http://capital-mercy.ru,1_9,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,html Благотворительный фонд Столица милосердия...,[http://capital-mercy.ru/fotogalereya/album/de...,[http://capital-mercy.ru/fotogalereya/album/de...
3,Москва,Российский общественный благотворительный фонд...,7704058560,1998-06-05,"г Москва, р-н Соколиная Гора, ул Щербаковская,...",1,1,http://www.rbfondveteranov.ru/,1_4,5_2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD XHTML 1.0 Strict//EN""...","[http://www.rbfondveteranov.ru/tyrizm, rbfondv...","[http://www.rbfondveteranov.ru/tyrizm, rbfondv..."
4,Москва,"Благотворительный фонд ""Фонд Валерия Гергиева""",7709402778,2003-01-27,"г Москва, р-н Арбат, ул Молчановка Б., д 23 к 2",0,0,http://gergiev-charity.com/rus/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD HTML 4.01 Transitiona...","[http://gergiev-charity.com/rus/rus/contact/, ...","[http://gergiev-charity.com/rus/rus/contact/, ..."
5,Москва,Международный Благотворительный Фонд Владимира...,7710012370,1994-05-20,"г Москва, Пресненский р-н, Шведский тупик, ...",1,0,http://spivakov.ru/fund/contacts/,4_2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,html Контактная информация и реквизиты | Фонд ...,"[http://spivakov.ru/fund/contacts/events/, mai...","[http://spivakov.ru/fund/contacts/events/, htt..."
6,Москва,Благотворительный фонд поддержки детей с особе...,7710479817,2012-05-02,"г Москва, Пресненский р-н, пер Кондратьевский ...",1,1,http://yaest.ru/,1_7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,"html Главная | Я есть <script src=""http://ajax...","[about/news/festival-molodezhi-i-studentov, ht...",[http://yaest.ru/bytags/?tag=Удивительные елки...
7,Москва,Благотворительный фонд помощи бездомным животн...,7716451494,2014-11-14,"г Москва, Ярославский р-н, Ярославское шоссе, ...",1,0,http://fond-kovcheg.ru,6_4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,html Фонд «Ковчег» | Поможем вместе window._wp...,"[http://fond-kovcheg.ru/view/2480, http://fond...","[http://fond-kovcheg.ru/view/2480, http://fond..."
8,Москва,Благотворительный фонд поддержки детей имени и...,7701064809,2011-03-22,"105082, ГОРОД МОСКВА, УЛИЦА ФРИДРИХА ЭНГЕЛЬСА,...",1,1,http://www.vfonde.ru/,1_7,1_2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,"html PUBLIC ""-//W3C//DTD XHTML 1.1//EN"" ""http:...",[http://www.vfonde.ru/index.php?vfonde=program...,[http://www.vfonde.ru/index.php?vfonde=program...
9,Москва,"Благотворительный Фонд ""Центр помощи беспризор...",7701295193,2002-03-27,"101000, ГОРОД МОСКВА, БУЛЬВАР ЧИСТОПРУДНЫЙ, 5/...",1,1,http://www.tppdetfond.ru/,1_3,1_7,...,NaN,NaN,NaN,NaN,NaN,NaN,1,html if(!window.BX)window.BX={};if(!window.BX....,"[tppdetfond.ru/o-fonde/popechitelskiy-sovet/, ...","[tppdetfond.ru/o-fonde/popechitelskiy-sovet/, ..."


In [102]:
new_df[0].replace([None],['None'],inplace=True)

In [112]:
sites = new_df["Ссылка на сайт"]
social = new_df[0]

In [116]:
social

0                                                                []
1                 [http://oppid.ru/zasedanie-rabochey-gruppyi-po...
2                 [http://capital-mercy.ru/fotogalereya/album/de...
3                 [http://www.rbfondveteranov.ru/tyrizm, rbfondv...
4                 [http://gergiev-charity.com/rus/rus/contact/, ...
5                 [http://spivakov.ru/fund/contacts/events/, htt...
6                 [http://yaest.ru/bytags/?tag=Удивительные елки...
7                 [http://fond-kovcheg.ru/view/2480, http://fond...
8                 [http://www.vfonde.ru/index.php?vfonde=program...
9                 [tppdetfond.ru/o-fonde/popechitelskiy-sovet/, ...
10                [kmfond.ru/index.php/novosti, kmfond.ru/index....
11                [http://mikhailrudyakfund.ru/#contacts, http:/...
12                [http://okrm.ru/kak-poluchat-zemlyu/, http://o...
13                [http://parkgeroev1945.ru/video/, http://parkg...
14                [http://sobr-zvezda.ru/%d0%b7%

In [113]:
social["Ссылка на сайт"] = social[0]
social.drop(columns=[0], inplace=True)
concat_dataset = pd.concat([sites, social], ignore_index=True)

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [114]:
concat_dataset

0                                http://greenstartup.ru/
1                                       http://oppid.ru/
2                                http://capital-mercy.ru
3                         http://www.rbfondveteranov.ru/
4                        http://gergiev-charity.com/rus/
5                      http://spivakov.ru/fund/contacts/
6                                       http://yaest.ru/
7                                 http://fond-kovcheg.ru
8                                  http://www.vfonde.ru/
9                              http://www.tppdetfond.ru/
10                                 http://www.kmfond.ru/
11                          http://mikhailrudyakfund.ru/
12                                       http://okrm.ru/
13                             http://parkgeroev1945.ru/
14                                http://sobr-zvezda.ru/
15                           http://osobyipeterburg.com/
16                                https://vityaz.spb.ru/
17                             

In [103]:
def list_to(df):
    # Drop empty rows
    sites = df[~dataset["Ссылка на сайт"].isnull()]
    sites.drop(columns=["Ссылка на соц сеть"], inplace=True)
    social = df[~df[0].isnull()]

# Merging social web-page to a web-site, adding new data samples 
    social["Ссылка на сайт"] = social[0]
    social.drop(columns=[0], inplace=True)
    concat_dataset = pd.concat([sites, social], ignore_index=True)
    
    concat_dataset["Ссылка на сайт"] = concat_dataset["Ссылка на сайт"].apply(parser.link_splitter)
    df.dropna(axis=0, subset=["Ссылка на сайт"], inplace=True)
    
    for i in range(new_df[0].map(len).max()):
        max_len = len(df)
        for index, row in df.iterrows():
            if len(row["Ссылка на сайт"])>1:
                sites = row["Ссылка на сайт"]
                last_site = sites.pop(-1)
                concat_dataset["Ссылка на сайт"].loc[index] = sites
                row["Ссылка на сайт"] = [last_site]
                concat_dataset.loc[max_len] = row
                max_len += 1

In [104]:
list_to(df)

/home/niwuro/Environments/WorkEnv/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match